# **_Stock Market Trend Analysis (2021–2025)_**
### Group Members:
- Farhan Haider
- Imran Khan
- Saad Farooq
****

### **Datasets:** Companies Listing.csv, Pakistan Stock Market Data 2021-2025.csv

In [2]:
import pandas as pd, numpy as np, math, os

In [3]:
import matplotlib.pyplot as plt, seaborn as sns

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm

plt.rcParams['figure.figsize'] = (12,5)
sns.set_theme(style="darkgrid")

In [5]:
# Load datasets
PRICES_CSV = "https://raw.githubusercontent.com/farhanhaider95/StockMarketTrendSystem/refs/heads/main/Pakistan%20Stock%20Market%20Data%202021-2025.csv"
COMPANIES_CSV = "https://raw.githubusercontent.com/farhanhaider95/StockMarketTrendSystem/refs/heads/main/Companies%20Listing.csv"

# read as strings to guard parsing issues
df = pd.read_csv(PRICES_CSV, dtype=str)
symbol_info = pd.read_csv(COMPANIES_CSV, dtype=str)

print("Raw rows:", len(df), "Company rows:", len(symbol_info))
df.head()


Raw rows: 476696 Company rows: 553


,DATE,SYMBOL,LDCP,OPEN,HIGH,LOW,CLOSE,CHANGE,CHANGE (%),VOLUME
0,1/1/2021,786,11.81,11.94,12.79,11.81,12.07,0.26,2.201524132,121000
1,1/1/2021,AABS,335.97,349,349,336.01,341.73,5.76,1.714438789,1400
2,1/1/2021,ABL,85.37,84.66,85.9,83.85,83.99,-1.38,-1.616492913,125500
3,1/1/2021,ABOT,755.37,756,756,745.01,750.95,-4.42,-0.585143704,2550
4,1/1/2021,ACPL,150.05,153,153.9,148.5,149.48,-0.57,-0.379873376,167100


### 2. Clean column names, parse dates, numeric conversions
- Normalize column names
- Parse date with `dayfirst=True` to handle `DD-MM-YY` and `MM/DD/YYYY`
- Convert numeric columns (remove commas/percent signs)
- Drop rows lacking essential values


In [38]:
# Normalize columns & parse date ---
# Normalize column names (strip & upper)
df.columns = df.columns.str.strip().str.upper()
symbol_info.columns = symbol_info.columns.str.strip().str.upper()

# Show columns
print("Price columns:", df.columns.tolist())
print("Symbol columns:", symbol_info.columns.tolist())

# Parse DATE robustly
df['DATE'] = pd.to_datetime(df['DATE'], dayfirst=True, errors='coerce')
df = df.dropna(subset=['DATE']).copy()

# Normalize SYMBOL
df['SYMBOL'] = df['SYMBOL'].str.strip().str.upper()
symbol_info['SYMBOL'] = symbol_info['SYMBOL'].str.strip().str.upper()

# Clean numeric columns and convert
numeric_cols = ['LDCP','OPEN','HIGH','LOW','CLOSE','CHANGE','CHANGE (%)','VOLUME']
# If 'CHANGE (%)' has percent sign, remove
if 'CHANGE (%)' in df.columns:
    df['CHANGE (%)'] = df['CHANGE (%)'].str.replace('%','', regex=False)
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].str.replace(',','', regex=False), errors='coerce')

# Drop rows without CLOSE (essential)
df = df.dropna(subset=['CLOSE']).copy()

# Sort for group operations
df = df.sort_values(['SYMBOL','DATE']).reset_index(drop=True)
df.info()


Price columns: ['DATE', 'SYMBOL', 'LDCP', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'CHANGE', 'CHANGE (%)', 'VOLUME']
Symbol columns: ['SYMBOL', 'NAME', 'SECTOR', 'STATUS']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188123 entries, 0 to 188122
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   DATE        188123 non-null  datetime64[ns]
 1   SYMBOL      188123 non-null  object        
 2   LDCP        188123 non-null  float64       
 3   OPEN        188123 non-null  float64       
 4   HIGH        188123 non-null  float64       
 5   LOW         188123 non-null  float64       
 6   CLOSE       188123 non-null  float64       
 7   CHANGE      188123 non-null  float64       
 8   CHANGE (%)  188120 non-null  float64       
 9   VOLUME      188123 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int64(1), object(1)
memory usage: 14.4+ MB
